In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [2]:
paper_pmid = 31854076
paper_name = 'novarina_chang_2019' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [29]:
original_data = pd.read_excel('raw_data/acel13084-sup-0001-files1.xlsx', sheet_nam='Filtered', header=8, nrows=4898-9)

In [30]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4889 x 26


In [31]:
original_data['Systematic_Name'] = original_data['Systematic_Name'].astype(str)

In [32]:
# Eliminate all white spaces & capitalize
original_data['Systematic_Name'] = clean_orf(original_data['Systematic_Name'])

In [33]:
original_data.loc[original_data['Systematic_Name']=='YLR287-A','Systematic_Name'] = 'YLR287C-A'

In [34]:
# Make sure everything translated ok
t = looks_like_orf(original_data['Systematic_Name'])

In [26]:
print(original_data.loc[~t,])

     Systematic_Name Common_Name  total_colonies  total_escapers  \
3033           BLANK       BLANK              18               4   

      percentage_escapers  pval_percent_escapers  median_size_escapers  \
3033            22.222222               0.901423                 596.0   

      pval_size_escapers  escaper_size_1  escaper_size_2  ...  escaper_size_9  \
3033             0.35224           829.0           463.0  ...             NaN   

      escaper_size_10  escaper_size_11  escaper_size_12  escaper_size_13  \
3033              NaN              NaN              NaN              NaN   

      escaper_size_14  escaper_size_15  escaper_size_16  escaper_size_17  \
3033              NaN              NaN              NaN              NaN   

      escaper_size_18  
3033              NaN  

[1 rows x 26 columns]


In [36]:
original_data.drop(index=original_data.loc[~t,].index, inplace=True)

In [37]:
print(original_data.shape)

(4888, 26)


In [38]:
original_data.head()

,Systematic_Name,Common_Name,total_colonies,total_escapers,percentage_escapers,pval_percent_escapers,median_size_escapers,pval_size_escapers,escaper_size_1,escaper_size_2,...,escaper_size_9,escaper_size_10,escaper_size_11,escaper_size_12,escaper_size_13,escaper_size_14,escaper_size_15,escaper_size_16,escaper_size_17,escaper_size_18
0,YHL033C,RPL8A,18,18,100.0,9.988568e-12,4880.0,2.767236e-13,5366.0,5073.0,...,5299.0,4763.0,5113.0,4561.0,4503.0,4649.0,4191.0,4265.0,4512.0,4860.0
1,YGL115W,SNF4,18,18,100.0,9.988568e-12,4569.0,2.873003e-13,4927.0,4458.0,...,4306.0,3759.0,3984.0,4151.0,4789.0,4787.0,4620.0,3483.0,4015.0,5051.0
2,YLR399C,BDF1,18,18,100.0,9.988568e-12,4372.5,2.873003e-13,3913.0,5318.0,...,4386.0,4120.0,4777.0,4359.0,4071.0,4040.0,4412.0,4099.0,4439.0,4177.0
3,YEL044W,IES6,18,18,100.0,9.988568e-12,4059.5,3.569460e-13,4148.0,4493.0,...,4189.0,7256.0,3640.0,3080.0,3032.0,7127.0,3529.0,3396.0,3245.0,3734.0
4,YPR099C,-,18,18,100.0,9.988568e-12,3930.5,3.166931e-13,4719.0,4925.0,...,3569.0,3686.0,3835.0,3875.0,3717.0,3147.0,3986.0,3836.0,4393.0,4286.0


In [39]:
original_data.set_index('Systematic_Name', inplace=True)

In [40]:
# Get the relevant columns
original_data = original_data.loc[:, 'percentage_escapers'].to_frame()

In [41]:
print(original_data.shape)

(4888, 1)


In [42]:
# If the same strain is present more than once, average its values
data = original_data.groupby(original_data.index.values).mean()

In [43]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4640 x 1


# Prepare the final dataset

In [44]:
dataset_ids = [16401]

In [45]:
datasets = datasets.reindex(index=dataset_ids)

In [46]:
# Create row index
data.index.name='orf'
data.columns = datasets['name']

# Print out

In [48]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [49]:
from IO.save_data_to_db2 import *

In [50]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [51]:
save_data_to_db(data, paper_pmid)